# Step 2: Generate data samples

In [ ]:
import yaml

with open("amplitude_model_helicity.yml") as input_file:
    recipe = yaml.load(input_file.read(), Loader=yaml.SafeLoader)

## 2.1 Define kinematics of the particle reaction

In [ ]:
from tensorwaves.physics.helicity_formalism.kinematics import HelicityKinematics
from tensorwaves.physics.particle import extract_particles

kin = HelicityKinematics.from_recipe(recipe)
particles = extract_particles(recipe)
print("Initial state mass:", kin.reaction_kinematics_info.initial_state_masses)
print("Final state masses:", kin.reaction_kinematics_info.final_state_masses)
print("Involved particles:", list(particles))

## 2.2 Generate phase space sample

In [ ]:
from tensorwaves.data.generate import generate_phsp

phsp_sample = generate_phsp(300000, kin)
phsp_sample.shape

As you can see, the phase-space sample is a three-dimensional array: 300.000
events of four-momentum tuples for three particles.

## 2.3 Generate intensity-based sample

In [ ]:
from tensorwaves.physics.helicity_formalism.amplitude import IntensityBuilder

builder = IntensityBuilder(particles, kin, phsp_sample)
intensity = builder.create_intensity(recipe)

We first decrease the debugging level a bit to avoid some redundant warnings.

In [ ]:
import logging

logging.disable(logging.WARNING)
logging.getLogger().setLevel(logging.INFO)

That's it, now we have enough info to create an intensity-based data sample.
Notice how the structure is the sample as the phase-space sample we saw
before:

In [ ]:
from tensorwaves.data.generate import generate_data

data_sample = generate_data(30000, kin, intensity)
data_sample.shape

## 2.4 Visualize kinematic variables

In [ ]:
phsp_set = kin.convert(phsp_sample)
data_set = kin.convert(data_sample)
list(data_set.keys())

In [ ]:
import pandas as pd

data_frame = pd.DataFrame(data_set)
phsp_frame = pd.DataFrame(phsp_set)
data_frame

In [ ]:
phsp_frame["mSq_3_4"].hist(bins=100)

In [ ]:
data_frame["mSq_3_4"].hist(bins=100, alpha=0.5, density=True)

## 2.5 Export data sets

In [ ]:
import numpy as np

np.save("data_sample", data_sample)
np.save("phsp_sample", phsp_sample)